In [1]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
from alphamind.api import *
from PyFin.api import *

from matplotlib import rc
from matplotlib import pyplot as plt

plt.style.use("seaborn-poster")
rc('font', **{'family': 'Microsoft Yahei', 'size': 10})
rc('mathtext', **{'default': 'regular'})
rc('legend', **{'frameon': False})

## 1. 准备因子历史数据
-----------------------

In [4]:
factor_name = 'EMA5D'
neutralized_styles = risk_styles + industry_styles
start_date = '2020-01-02'
end_date = '2020-02-21'
engine = SqlEngine(os.environ['DB_URI'])
universe_name = 'hs300'
freq = '5b'

In [5]:
universe = Universe(universe_name)
dates = makeSchedule(start_date, end_date, freq, calendar='china.sse')
factors = engine.fetch_factor_range(universe, factor_name, dates=dates)
risk_cov, risk_factors = engine.fetch_risk_model_range(universe, dates=dates)
dx_returns = engine.fetch_dx_return_range(universe, dates=dates, horizon=map_freq(freq))
total_data = pd.merge(factors, risk_factors, on=['trade_date', 'code'])
total_data = pd.merge(total_data, dx_returns, on=['trade_date', 'code'])
total_data.dropna(inplace=True)

## 2. 计算历史IC序列
----------------------

In [8]:
neutralized_factors = neutralize(total_data[neutralized_styles].values.astype(float),
                                 total_data[factor_name].values,
                                 groups=total_data['trade_date'])
total_data['neutralized_factors'] = neutralized_factors

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Int64Index([253], dtype='int64'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [9]:
ic_series = total_data.groupby('trade_date').apply(lambda x: np.corrcoef(x['neutralized_factors'], x['dx'])[0, 1])
ic_series.rolling(window=7).mean().plot(figsize=(14, 7))

KeyError: 'neutralized_factors'

## 3. 行业内IR计算
---------------------

In [10]:
industry_category = engine.fetch_industry_range(universe, dates=dates)
total_data = pd.merge(total_data, industry_category, on=['trade_date', 'code']).dropna()

In [11]:
industry_ic = total_data.groupby(['trade_date', 'industry']).apply(lambda x: np.corrcoef(x['neutralized_factors'], x['dx'])[0, 1])

KeyError: 'neutralized_factors'

In [12]:
(industry_ic.groupby(level=1).mean() / industry_ic.groupby(level=1).std()).sort_values(ascending=False).plot(kind='bar', figsize=(14, 7))

NameError: name 'industry_ic' is not defined